In [10]:
import numpy as np
from time import perf_counter as tpc

from teneva import getter, cross, rand, erank

np.random.seed(42)

In [31]:
def check(X_tst, Y_tst, Y, get, name):
    """Проверяет точность модели на заданном наборе данных."""
    t = tpc()
    Z = np.array([get(x) for x in X_tst])
    t = tpc() - t
    e = np.linalg.norm(Z - Y_tst) / np.linalg.norm(Y_tst)

    print(f'------------------ {name}')
    print(f'Check time (sec) : {t:-8.3f}')
    print(f'Time 1-calc (ms) : {t/len(Z) * 1000:-8.3f}')
    print(f'Relative error   : {e:-8.2e}')
    print(f'Erank is         : {erank(Y):-8.2f}')

In [32]:
N = [3, 4, 5, 7]    # Форма тензора
d = len(N)          # Размерность задачи
K = 1000            # Количество обучающих примеров
M = 10000           # Количество проверочных примеров

In [45]:
def f(x):
    y = np.sum([x[i]**(i+1) for i in range(d)])
    y = np.sin(y)**2 / (1 + y**2)
    return  y

def f_vect(X): # "Наивно векторизованная" модельная функция
    return np.array([f(x) for x in X])

In [46]:
X_tst = np.vstack([np.random.choice(N[i], M) for i in range(d)]).T
Y_tst = np.array([f(x) for x in X_tst])

In [47]:
Y0 = rand(N, 2)

In [48]:
# Здесь мы используем maxvol из пакета ttpy:
import tt
from tt.cross.rectcross.rect_maxvol import maxvol as base_maxvol
from tt.cross.rectcross.rect_maxvol import rect_maxvol as base_rect_maxvol

In [49]:
t = tpc()

maxvol = base_maxvol
rect_maxvol = base_rect_maxvol
Y = cross(f_vect, Y0, nswp=3, kickrank=2, rf=1.5, mv=maxvol, rmv=rect_maxvol)

print(f'Build time (sec) : {tpc() - t:-8.3f}')

get = getter(Y)
check(X_tst, Y_tst, Y, get, 'Cross own')

Build time (sec) :    0.100
------------------ Cross own
Check time (sec) :    0.451
Time 1-calc (ms) :    0.045
Relative error   : 1.23e-16
Erank is         :     9.09


In [51]:
# Считаем то же самое, используя ttpy:
import tt                                   # Только для Cross
from tt.cross.rectcross import cross as cr  # Только для Cross

def solver_cro(f, Y0, nswp=10, e=1.E-6):
    Y0 = tt.tensor.from_list(Y0).round(e)
    Y = cr(myfun=f, x0=Y0, nswp=nswp, eps=e,
              eps_abs=0., kickrank=2, rf=1.5,
              verbose=True, stop_fun=None, approx_fun=None)
    return tt.tensor.to_list(Y) # (Y.round(e))       

t = tpc()
Yr_cro = solver_cro(f_vect, Y0, nswp=3, e=1.E-10)

print(f'Build time (sec) : {tpc() - t:-8.3f}')

get_cro = getter(Yr_cro)
check(X_tst, Y_tst, Yr_cro, get_cro, 'Cross')

swp: 0/2 er_rel = 3.9e+01 er_abs = 3.4e+01 erank = 5.8 fun_eval: 333
swp: 1/2 er_rel = 2.0e-01 er_abs = 1.7e-01 erank = 7.7 fun_eval: 1187
swp: 2/2 er_rel = 3.3e-04 er_abs = 2.7e-04 erank = 9.1 fun_eval: 2471
Build time (sec) :    0.120
------------------ Cross
Check time (sec) :    0.472
Time 1-calc (ms) :    0.047
Relative error   : 1.23e-16
Erank is         :     9.09


# Below is old code !!!

In [ ]:
from code_ import rand_
from code_ import mul_
from code_ import getter_

In [ ]:
from code import getter

In [ ]:
def tcheck(func, K, title=''):
    t = tpc()

    for k in range(K):
        func()

    t = tpc() - t
    if title:
        print(f'>>> {title}')
    print(f'Total time    (sec ) : {t:-10.4f}')
    print(f'Time for 1 op (msec) : {t/K:-10.6f}')

In [ ]:
N = [10] * 50
d = len(N)
A = rand_(N, 7)
B = rand_(N, 6)
C = mul_(A, B)

In [ ]:
get_ = getter_(C)
get = getter(C)

In [ ]:
x = np.zeros(d, dtype=int)
get(x)

In [ ]:
x = np.zeros(d, dtype=int)
get_(x)

In [ ]:
tcheck(lambda: get_(x), 1000, 'get | Numpy')

In [ ]:
tcheck(lambda: get(x), 1000, 'get | Numba')

In [ ]:
A_tt = tt.tensor.from_list(A)
B_tt = tt.tensor.from_list(B)
C_tt = A_tt * B_tt

tcheck(lambda: C_tt[x], 1000, 'get | Ttpy')

In [ ]:
tcheck(lambda: mul_(A, B), 1000, 'mul | Numpy')

In [ ]:
A_tt = tt.tensor.from_list(A)
B_tt = tt.tensor.from_list(B)

tcheck(lambda: A_tt * B_tt, 1000, 'mul | Ttpy')

In [ ]:
A_tt = tt.tensor.from_list(A)
B_tt = tt.tensor.from_list(B)
C_tt = A_tt * B_tt

In [ ]:
if False:
    X1 = tt.tensor.from_list(C).full()
    X2 = C_tt.full()
    e = np.max(np.abs(X1 - X2))
    print(f'Error {e:-8.2e}')

In [ ]:
t = tpc()

K = 1000
for k in range(K):
    C_tt = A_tt * B_tt
    
t = tpc() - t
print(f'Total time    (sec ) : {t:-10.4f}')
print(f'Time for 1 op (msec) : {t/K:-10.6f}')

In [ ]:
t = tpc()

K = 1000
for k in range(K):
    C = mul(A, B)
    
t = tpc() - t
print(f'Total time    (sec ) : {t:-10.4f}')
print(f'Time for 1 op (msec) : {t/K:-10.6f}')

In [ ]:
t = tpc()

K = 1000
for k in range(K):
    C = mul_nb(A, B)
    
t = tpc() - t
print(f'Total time    (sec ) : {t:-10.4f}')
print(f'Time for 1 op (msec) : {t/K:-10.6f}')